In [ ]:
import src.acquire
import src.prepare
import src.explore
import src.model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Addidtional Reporting Outside of Model

1. What state and county are these properties in?
2. Distribution of tax rates for each county
3. Tax Rate for each property
4. How much does the tax rate vary by county?
5. What tax rate do the bulk of the properties sit around?

In [ ]:
path='zillow_df.csv'

Df, X_train_explore, \
    X_train_scaled, y_train, \
    X_validate_scaled, y_validate, \
    X_test_scaled, y_test = src.prepare.wrangle_zillow(path)

X_train_scaled.shape, X_validate_scaled.shape, X_test_scaled.shape

**Use the total datafame**

This infomation has been requested by the customer for the total properties with a transaction from May or June 2017.

Dropped observations
    - 1 dropped because there was no tax amount for this property
    - 13 dropped as outliers because property tax rate was above 10%
    
It 2019 the highest county tax rate in the US was 6.6% so values above 10% in 2017 were determined to be errors and dropped from the dataframe.  
https://www.attomdata.com/news/market-trends/figuresfriday/top-10-u-s-counties-with-the-greatest-effective-tax-rates/

In [ ]:
Df.head()

In [ ]:
X_train_explore.head()

In [ ]:
X_train_scaled.head()

In [ ]:
# get full dataset within the date range May - June 2017
tax = src.acquire.get_zillow_data()

In [ ]:
tax.shape

In [ ]:
tax.columns

In [ ]:
# drop columns not related to tax calculation or location
taxdf = tax[['fips', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
            'taxamount', 'transactiondate', 'latitude', 'longitude', 'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc',
                      'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip']]


In [ ]:
taxdf.isnull().sum()

In [ ]:
# drop 'structuretaxvaluedollarcnt', 'propertyzoningdesc', 'regionidcity','regionidneighborhood'  = too many nulls
# drop 'landtaxvaluedollarcnt': 'taxvaluedollarcnt' should be the total tax which is what we want
# drop single record that is missing taxamount, one out of +20,000 should not impact overall calculations
# drop 14 records that are missing regionidzip, 14 out of +20,000 should not impact overall calculations
taxdf = taxdf.drop(columns=['regionidneighborhood', 'structuretaxvaluedollarcnt', 'propertyzoningdesc', 'regionidcity', 'landtaxvaluedollarcnt'])
taxdf = taxdf.dropna(axis=0)
taxdf.head()

In [ ]:
taxdf.isnull().sum()

In [ ]:
taxdf['tax_rate'] = taxdf.taxamount/taxdf.taxvaluedollarcnt
taxdf.head()

In [ ]:
taxdf.tax_rate.describe()
# note the max is way out of range, drop?

In [ ]:
taxdf.tax_rate.median()

In [ ]:
plt.figure(figsize=(16, 7))
sns.boxplot(taxdf.tax_rate)

In [ ]:
outliers = taxdf[taxdf.tax_rate > .1]
outliers
# note from this article that in some parts of US the effective tax rate for 2019 was as high as 6%
# 

In [ ]:
# drop these specific outliers because the variance caused skews the data too far and 13 observations is small amount
# also domain research indicate that as of 2019 6.6% is highest tax rate in any US county
# working on the assumption that if State is requested info 
taxdf = taxdf.drop(labels=[1600, 1628, 5099, 5969, 8109, 8407, 8521, 8849, 11562, 12430, 14313, 20313, 21502], axis=0)

In [ ]:
out_test = taxdf[taxdf.tax_rate > .1]
out_test

In [ ]:
taxdf.shape

In [ ]:
taxdf.groupby('regionidcounty')[['tax_rate']].min()

In [ ]:
taxdf.groupby('regionidcounty')[['tax_rate']].max()

In [ ]:
taxdf.regionidcounty.value_counts()

In [ ]:
# After removal of initial outliers
plt.figure(figsize=(16, 7))
sns.boxplot(taxdf.tax_rate)

In [ ]:
# total distribution
plt.figure(figsize=(16, 7))
sns.distplot(taxdf.tax_rate, rug=False)


In [ ]:
# how to find state? what is fips column?
# https://en.wikipedia.org/wiki/FIPS_county_code

In [ ]:
x = taxdf.latitude
y = taxdf.longitude
plt.scatter(x, y)
plt.show()

In [ ]:
taxdf.fips.value_counts()

### per County FIPS Codes list

06037 = Los Angles county

06059 = Orange county

06111 = Ventura county

https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697

In [ ]:
https://seaborn.pydata.org/examples/histogram_stacked.html

https://seaborn.pydata.org/examples/kde_ridgeplot.html

In [ ]:
X_train_explore.rename(columns = {'Unnamed: 0':'delete', 'id.1':'delete1'}, inplace = True)
X_train_explore.drop(columns = ['delete','delete1','yearbuilt'], inplace = True)
X_train_scaled.rename(columns = {'Unnamed: 0':'delete', 'id.1':'delete1'}, inplace = True)
X_train_scaled.drop(columns = ['delete','delete1'], inplace = True)
X_validate_scaled.rename(columns = {'Unnamed: 0':'delete', 'id.1':'delete1'}, inplace = True)
X_validate_scaled.drop(columns = ['delete','delete1'], inplace = True) 
X_test_scaled.rename(columns = {'Unnamed: 0':'delete', 'id.1':'delete1'}, inplace = True)


In [ ]:
X_train_scaled.head()

In [ ]:
X_train_explore.info()

In [ ]:
X_train_explore.head()

In [ ]:
Df.head()

In [ ]:
Df.shape

In [ ]:
Df['taxrte'] = Df.

In [ ]:
df = pd.DataFrame()
df["stuff"] = ["a", "b", "c", "d", "e" , "f", "g"]
rows_to_remove = [2, 3, 4]
df = df[~df.index.isin(rows_to_remove)]

## Notes

1. I already uploaded my env.py file so need need to upload yours.  We only need one to access the data.


2. I already uploaded a .gitignore so also no worries there


3. Data is ready to go, i want to engeneer a feature that combines bed and bath, but get a weird error when I try.    we can trouble shoot after mvp, but code can be found in .prepare file


4. We have 3 categorical variables  and 14 numeric
   - "regionidzip"
   - "propertylandusetypeid"
   - 'yearbuilt'
   

5. id.1 and Unmaed:0 showd up we also need to delete them

***

## What Data do we need in our initial SQL querry?


What is a single unit property?

By James Chen Updated Sep 11, 2020 What Is a Housing Unit?

The term housing unit refers to a single unit within a larger structure that can be used by an individual or household to eat, sleep, and live. The unit can be in any type of residence such as a house, apartment, mobile home, or may also be a single unit in a group of rooms. Essentially, a housing unit is deemed to be a separate living quarter where the occupants live and eat separately from other residents of the structure or building. They also have direct access from the building's exterior or through a common hallway.

    https://www.investopedia.com/terms/h/housingunits.asp

Properties in the Database: Based on the above definition some categories do not fit brief

Propertylandusetypeid | propertylandusedesc

  No        31           Commercial/Office/Residential Mixed Used  (not a residence)
  No        46           Multi-Story Store                         (not a residence)
  No        47           Store/Office (Mixed Use)                  (not a residence)
            246          Duplex (2 Units, Any Combination)
            247          Triplex (3 Units, Any Combination)
            248          Quadruplex (4 Units, Any Combination)
            260          Residential General
            261          Single Family Residential
            262          Rural Residence
            263          Mobile Home
            264          Townhouse
            265          Cluster Home
            266          Condominium
  No        267          Cooperative                               (become shareholder not owner)
            268          Row House
            269          Planned Unit Development
  No        270          Residential Common Area                   (propterty feature)
  No        271          Timeshare                                 (become shareholder not owner)
            273          Bungalow
            274          Zero Lot Line
            275          Manufactured, Modular, Prefabricated Homes
            276          Patio Home
            279          Inferred Single Family Residential
  No        290          Vacant Land - General                     (not a residence)
  No        291          Residential Vacant Land                   (not a residence)



## Feature Selection (my thoughts and why)



What Data do we need in our inital df? (20,931 Housing Units)

    parcelid 20931 non-null int64 Listing Number - Drop for Explore
    
    
    id 20931 non-null int64 Listing ID - Drop for Explore
    
    
    airconditioningtypeid 6779 non-null float64 Too Many null-values - Drop for Explore
    
    
    architecturalstyletypeid 52 non-null float64 Too Many null-values - Drop for Explore
    
    
    basementsqft 16 non-null float64 16 non-values - Drop for Explore
    
    
    bathroomcnt 20931 non-null float64 Use
    
    
    bedroomcnt 20931 non-null float64 Use - Combine bath/bed (feature engeneering)
    
    
    buildingclasstypeid 0 non-null object All Values Null - Drop for Explore
    
    
    buildingqualitytypeid 13257 non-null float64 Too Many null-values - Drop for Explore
    
    
    calculatedbathnbr 20771 non-null float64 Repeat of barthromcnt - Drop for Explore
    
    
    decktypeid 174 non-null float64 Too Many null-values - Drop for Explore
    
    
    finishedfloor1squarefeet 1738 non-null float64 Repeat Column - Drop for Explore
    
    
    calculatedfinishedsquarefeet 20868 non-null float64 Use - Drop null
    
    
    finishedsquarefeet12 20024 non-null float64 Repeat Column - Drop for Explore
    
    
    finishedsquarefeet13 17 non-null float64 Repeat Column - Drop for Explore
    
    
    finishedsquarefeet15 736 non-null float64 Repeat Column - Drop for Explore
    
    
    finishedsquarefeet50 1738 non-null float64 Repeat Column - Drop for Explore
    
    
    finishedsquarefeet6 91 non-null float64 Repeat Column - Drop for Explore
    
    
    fips 20931 non-null float64 Repeat Column - Drop for Explore
    
    
    fireplacecnt 2422 non-null float64 Use - change null to 0
    
    
    fullbathcnt 20771 non-null float64 Repeat of bathroom - Drop for Explore
    
    
    garagecarcnt 7075 non-null float64 Use - Rename as garage, change null to 0
    
    
    garagetotalsqft 7075 non-null float64 - garagesqft verifys that they exist
    
    
    hashottuborspa 461 non-null float64 Use - change null to 0, for no ht or spa
    
    
    heatingorsystemtypeid 13285 non-null float64 Too Many null-values - Drop for Explore
    
    
    latitude 20931 non-null float64 Repeat Column - Drop for Explore
    
    
    longitude 20931 non-null float64 Repeat Column - Drop for Explore
    
    
    lotsizesquarefeet 18742 non-null float64 Too Large for Modeling, Scaling?
    
    
    poolcnt 4496 non-null float64 Use - change null to 0, for no pool
    
    
    poolsizesum 251 non-null float64 Repeat Column - Drop for Explore
    
    
    pooltypeid10 121 non-null float64 Repeat Column - Drop for Explore
    
    
    pooltypeid2 340 non-null float64 Repeat Column - Drop for Explore
    
    
    pooltypeid7 4154 non-null float64 Repeat Column - Drop for Explore
    
    
    propertycountylandusecode 20931 non-null object Repeat Column - Drop for Explore
    
    
    propertylandusetypeid 20931 non-null float64 Use - Categories
    
    
    propertyzoningdesc 13437 non-null object Too Many null-values - Drop for Explore
    
    
    rawcensustractandblock 20931 non-null float64 **Repeat info(zip)
    
    
    regionidcity 20503 non-null float64 Repeat info(zip) - Drop for Explore
    
    
    regionidcounty 20931 non-null float64 Repeat info(zip) - Drop for Explore
    
    
    regionidneighborhood 8443 non-null float64 Too Many null-values - Drop for Explore
    
    
    regionidzip 20916 non-null float64 Use - latered to categorical
    
    
    roomcnt 20931 non-null float64 Use
    
    
    storytypeid 16 non-null float64 Too Many null-values - Drop for Explore
    
    
    threequarterbathnbr 2800 non-null float64 Repeat info(bathroom) - Drop for Explore
    
    
    typeconstructiontypeid 56 non-null float64 Too Many null-values - Drop for Explore
    
    
    unitcnt 13476 non-null float64 Repeat info() - Drop for Explore
    
    
    yardbuildingsqft17 701 non-null float64 Too Many null-values - Drop for Explore
    
    
    yardbuildingsqft26 25 non-null float64 Too Many null-values - Drop for Explore
    
    
    yearbuilt 20850 non-null float64 Use - Drop null values
    
    
    numberofstories 4917 non-null float64 Too Many null-values - Drop for Explore
    
    
    fireplaceflag 51 non-null float64 Repeat info(firepls) - Drop for Explore
    
    
    structuretaxvaluedollarcnt 20897 non-null float64 Correlates w/Target - Drop for Explore
    
    
    taxvaluedollarcnt 20930 non-null float64 Target Variable
    
    
    assessmentyear 20931 non-null float64 Filtered in SQL - Drop for Explore
    
    
    landtaxvaluedollarcnt 20930 non-null float64 Correlates w/Target - Drop for Explore
    
    
    taxamount 20931 non-null float64 Correlates w/Target - Drop for Explore
    
    
    taxdelinquencyflag 703 non-null object Correlates w/Target - Drop for Explore
    
    
    taxdelinquencyyear 703 non-null float64 Correlates w/Target - Drop for Explore
    
    
    censustractandblock 20852 non-null float64 Repeat Column - Drop for Explore
    
    
    id 20931 non-null int64 Repeat Column - Drop for Explore
    
    
    logerror 20931 non-null float64 Calculation - Drop for Explore
    
    
    transactiondate 20931 non-null object Filtered in SQL - Drop for Explore

***

## Final Feature Selection (Corey)

**12 Numeric Features**
- bathroomcnt                    
- bedroomcnt                     
- calculatedfinishedsquarefeet   
- fireplace                      
- garage                        
- hottub_spa                    
- lotsizesquarefeet              
- poolcnt                        
- roomcnt                        
- yearbuilt                      
- zip                           
- useid  
- **Hopefull 13th Combination of bed and bath**


**2 Categorical Features**
 - zip
 - useid

***

## SQL Querries

In [ ]:


# Code to acquire data
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279');
'''

# Returns 20,931 entries between May and June 2017
# Data represents single unit properties



In [ ]:


# Code to return number of houses per county

'''
select regionidcounty, count(*)
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
group by regionidcounty
order by regionidcounty;
'''

# Returns 3 counties (1286, 2061, 3101)



In [ ]:


# Code to return houses per county (Group by) for tax rate distribution
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
order by regionidcounty;
'''



In [ ]:
# Breaks down houses per county in to cities per county and zipcodes
'''
select regionidcity, regionidcounty, regionidzip
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
group by regionidcity, regionidcounty, regionidzip
order by regionidzip;
'''
# Returns a list of zip codes and city id numbers per county. Necessary to id **STATE**

In [ ]:


# All of the zipcodes and counts for that zipcode in for the dataset
'''
select count(*), regionidzip
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
group by regionidzip
order by regionidzip;
'''
# Returns 385 unique zip codes



In [ ]:


# All of the properties in county code 1286
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
and regionidcounty = 1286;
'''
# Returns 5640 values



In [ ]:


# All of the properties in county code 2061
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
and regionidcounty = 2061;
'''
# Returns 1799 values



In [ ]:


# All of the properties in county code 3101
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
and regionidcounty = 3101;
'''
# Returns 13492 values



In [ ]:


# All of the properties in county code Null
'''
select *
from properties_2017
join unique_properties using (parcelid)
join predictions_2017 using (parcelid)
WHERE transactiondate between '2017-06-01' AND '2017-07-31'
AND propertylandusetypeid IN ('246','247','248','260','261','262','263','264','265','266','268','269','273','274','275','276','279') 
and regionidcounty is Null;
'''
# Returns 0 values

